<a href="https://colab.research.google.com/github/1kaiser/Media-Segment-Depth-MLP/blob/main/Segmentation_Depth_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## ***RUNNING MEDIAPIPE HOLISTIC***

In [ ]:
!wget -P /content/ --output-document=test.mp4 https://github.com/1kaiser/Media-Segment-Depth-MLP/releases/download/v0.2/video.mp4 

In [ ]:
!ls

In [ ]:
!python -m pip install mediapipe

In [ ]:
video_location = '/content/test.mp4'
import os

 
# Read images with OpenCV.
#images= None
image_dir = '/content/MEDIAPIPEinput/'
os.makedirs(image_dir, exist_ok=True)
image_dir_out = '/content/annotated_images'
os.makedirs(image_dir_out, exist_ok=True)
frame_rate = 24
!ffmpeg -y \
  -i {video_location} \
  -r {frame_rate} {image_dir}out_%09d.png

imgs_list = os.listdir(image_dir)
imgs_list.sort()
imgs_path = [os.path.join(image_dir, i) for i in imgs_list if i != 'outputs']


In [ ]:
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import mediapipe as mp
# mp_holistic = mp.solutions.holistic
mp_pose = mp.solutions.pose
!rm -r {image_dir}.ipynb_checkpoints

# Run MediaPipe Pose with `enable_segmentation=True` to get pose segmentation.
with mp_pose.Pose(static_image_mode=True, 
                          min_detection_confidence=0.2,
                          model_complexity=2, 
                          enable_segmentation=True,) as pose:
  temp_segmentation_mask =[]                        
  for name, image in enumerate(imgs_path):
    !rm -r {image_dir}.ipynb_checkpoints
    # Convert the BGR image to RGB and process it with MediaPipe Pose.
    image = cv2.imread(image)
    results = pose.process(image)

    # Draw pose segmentation.
    print(f'Pose segmentation of {name}:')
    annotated_image_pose = image.copy()
    red_img = np.zeros_like(annotated_image_pose, dtype=np.uint8)
    red_img[:, :] = (255,255,255)
    ###check if segmentation_mask exists or not ## if exists then ok Else use previous mask temporarily
    if results.segmentation_mask is None:
      print("true")
      results.segmentation_mask = temp_segmentation_mask[-1]
      temp_segmentation_mask.append(results.segmentation_mask)
    else:
      temp_segmentation_mask.append(results.segmentation_mask)
    ###End check if segmentation_mask exists or not ## if exists then ok Else use previous mask temporarily
    segm_2class = 0.0 + 1.0 * results.segmentation_mask
    segm_2class = np.repeat(segm_2class[..., np.newaxis], 3, axis=2)
    annotated_image_pose = annotated_image_pose * segm_2class + red_img * (1 - segm_2class)
    #resize_and_show(annotated_image)
    cv2.imwrite('%s/%s' %(image_dir_out, imgs_list[name]), annotated_image_pose)
    !rm -r {image_dir_out}.ipynb_checkpoints

# # Run MediaPipe Pose with `enable_segmentation=True` to get pose segmentation.
# with mp_holistic.Holistic(static_image_mode=False, 
#                           min_detection_confidence=0.5,
#                           model_complexity=2, 
#                           enable_segmentation=True,) as holistic:
#   temp_segmentation_mask =[]                        
#   for name, image in enumerate(imgs_path):
#     !rm -r {image_dir}.ipynb_checkpoints
#     # Convert the BGR image to RGB and process it with MediaPipe Pose.
#     image = cv2.imread(image)
#     results = holistic.process(image)

#     # Draw pose segmentation.
#     print(f'Pose segmentation of {name}:')
#     annotated_image_pose = image.copy()
#     red_img = np.zeros_like(annotated_image_pose, dtype=np.uint8)
#     red_img[:, :] = (255,255,255)
#     if results.segmentation_mask is None:
#       print("true")
#       results.segmentation_mask = temp_segmentation_mask[-1]
#       temp_segmentation_mask.append(results.segmentation_mask)
#     else:
#       temp_segmentation_mask.append(results.segmentation_mask)
#     segm_2class = 0.2 + 0.8 * results.segmentation_mask
#     segm_2class = np.repeat(segm_2class[..., np.newaxis], 3, axis=2)
#     annotated_image_pose = annotated_image_pose * segm_2class + red_img * (1 - segm_2class)
#     #resize_and_show(annotated_image)
#     cv2.imwrite('%s/%s' %(image_dir_out, imgs_list[name]), annotated_image_pose)
#     !rm -r {image_dir_out}.ipynb_checkpoints

In [11]:
frame_rate = 24
#create video 1
!ffmpeg \
  -framerate {frame_rate} \
  -pattern_type glob \
  -i {image_dir}'/*.png' \
  inputA.mp4
#create vodeo 2
!ffmpeg \
  -framerate {frame_rate} \
  -pattern_type glob \
  -i {image_dir_out}'/*.png' \
  inputB.mp4
#merge video 1 and video 2
!ffmpeg \
  -i inputA.mp4 -i inputB.mp4 \
  -filter_complex \
  "[0:v][1:v]vstack=inputs=2" \
  finalOutput.mp4
#add sound of main video to final output video
!ffmpeg \
  -i 'finalOutput.mp4' -i {video_location} \
  -c:v copy -c:a copy -map 0:v:0 -map 1:a:0 \
  OUTPUT_FILE.mp4

ffmpeg version 4.2.7-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-avresample --disable-filter=resample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librsvg --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --e

##***BoostingMonocularDepth code***

In [ ]:
# Code section 1
INPUT_FILES = 'input1'
OUTPUT_FILES = 'outputs_leres'
!mkdir -p {INPUT_FILES}
!mkdir -p {OUTPUT_FILES}

In [ ]:
!ls

In [ ]:
!ffmpeg -y -hwaccel cuvid \
  -i '/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/test.mp4' \
  -r 24 {INPUT_FILES}/out_%09d.png

In [ ]:
# Code section 2


# Clone git repo
!git clone https://github.com/1kaiser/BoostingMonocularDepth.git
!mkdir -p BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/
!cp "/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/latest_net_G.pth" "BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/"
!cp "/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/res101.pth" "BoostingMonocularDepth/"


In [ ]:
# # Code section 2


# # Clone git repo
# !git clone https://github.com/1kaiser/BoostingMonocularDepth.git

# # Downloading merge model weights 
# !mkdir -p /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/
# !gdown 1BbX2_NjrmrzRIB2CRJ-1rNRAjPABXPCh
# !mv latest_net_G.pth /content/BoostingMonocularDepth/pix2pix/checkpoints/mergemodel/

# # Downloading Midas weights
# #!gdown https://drive.google.com/uc?id=1nqW_Hwj86kslfsXR7EnXpEWdO2csz1cC
# #!mv model.pt /content/BoostingMonocularDepth/midas/

# # # Downloading LeRes weights 
# !gdown 1vni3Oh4y0S4eatz_u_aDnwDLwxBjUpCh
# !mv res101.pth /content/BoostingMonocularDepth/

In [ ]:
# Code section 3
%cd BoostingMonocularDepth/

check and delete  fiiles

In [ ]:
INPUT_FILES = '/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/drive/input1'
OUTPUT_FILES = "/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/drive/outputs_leres"

In [ ]:
%%writefile goDeleteHolistic.js
'use strict';

const fs = require('fs');
let d=0;
let darray1 = fs.readdirSync({INPUT_FILES});
let darray2 = fs.readdirSync({OUTPUT_FILES});
for (let index = 0; index < darray1.length; index++) {
    console.log(index)
    if (darray2.includes(darray1[index])) {
        console.log(d++)
        
        fs.unlinkSync({INPUT_FILES}+darray1[index])
    }
}

In [ ]:
!node goDeleteHolistic.js

In [ ]:
# Running the method using LeRes
!python run.py --Final --data_dir {INPUT_FILES} --output_dir  {OUTPUT_FILES}/ --depthNet 2

**Create Video Output**

In [ ]:
#create video 1
!ffmpeg \
-framerate 24 \
-pattern_type glob \
-i {INPUT_FILES}'/*.png' \
input0.mp4
#create vodeo 2
!ffmpeg \
-framerate 24 \
-pattern_type glob \
-i {OUTPUT_FILES}'/*.png' \
input1.mp4
#merge video 1 and video 2
!ffmpeg \
-i input0.mp4 -i input1.mp4 \
-filter_complex \
"[0:v][1:v]vstack=inputs=2" \
finalOutput.mp4
#add sound of main video to final output video
!ffmpeg \
-i 'finalOutput.mp4' -i 'm.mp4' \
-c:v copy -c:a copy -map 0:v:0 -map 1:a:0 \
OUTPUT_FILE.mp4

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/OUT/data/machine_learning_test_dataset/drive

In [ ]:
!pwd

**FFMPEG video to image**

##**MLP MODEL WITH POSITIONAL ENCODER**

In [ ]:
############<< LOADING DATASET >>#############

!cp "/content/drive/MyDrive/OUT/data/machine_learning_test_dataset/archive.zip" "/content/"
!unzip /content/archive.zip #unzipping the flower images from archive..
!mkdir -p /content/flowers/all
!cp /content/flowers/daisy/* /content/flowers/all
!cp /content/flowers/dandelion/* /content/flowers/all
!cp /content/flowers/rose/* /content/flowers/all
!cp /content/flowers/sunflower/* /content/flowers/all
!cp /content/flowers/tulip/* /content/flowers/all

##############$$$$$$$$$$$$$$<<< IMPORTING LIBRARIES >>>$$$$$$$$$$$$$$$$################
#✅
import jax
import jax.numpy as jnp

!python -m pip install -q -U flax
import flax
import optax
from typing import Any

from jax import lax
import flax.linen as nn
from flax.training import train_state
##########################################<< FFT PSEUDO SIN COS POSITIONAL ENCODER >>#########################################

positional_encoding_dims = 6  # Number of positional encodings applied

def positional_encoding(inputs):
    print("positional_encoding start")
    batch_size, _ = inputs.shape;print(inputs.shape)
    inputs_freq = jax.vmap(lambda x: inputs * 2.0 ** x)(jnp.arange(positional_encoding_dims));print(inputs_freq.shape)
    x = jnp.stack([jnp.sin(inputs_freq), jnp.cos(inputs_freq)]);print(x.shape)
    x = x.swapaxes(0, 2);print(x.shape)
    x = x.reshape([batch_size, -1]);print(x.shape)
    x = jnp.concatenate([inputs, x], axis=-1);print(x.shape)
    print("positional_encoding end")
    return x
##########################################<< MLP MODEL >>#########################################

apply_positional_encoding = True # Apply posittional encoding to the input or not
num_dense_layers = 8 # Number of dense layers in MLP
dense_layer_width = 256 # Dimentionality of dense layers' output space 

class MLPModel(nn.Module):
    dtype: Any = jnp.float32
    precision: Any = lax.Precision.DEFAULT
    apply_positional_encoding: bool = apply_positional_encoding
    @nn.compact
    def __call__(self, input_points):
        x = positional_encoding(input_points) if self.apply_positional_encoding else input_points
        print("network model start")
        print(x.shape)
        for i in range(num_dense_layers):
            x = nn.Dense(
                dense_layer_width,
                dtype=self.dtype,
                precision=self.precision
                )(x)
            x = nn.relu(x)
            x = jnp.concatenate([x, input_points], axis=-1) if i == 4 else x
            print(x.shape)
  
        x = nn.Dense(1, dtype=self.dtype, precision=self.precision)(x)
        print(x.shape)
        print("network model end")
        return x
##########################################<< MLP MODEL >>#########################################
     
##########################################<< CREATE MODEL INITIAL STATE FILLING WITH WEIGHTS >>#########################################

def init_train_state(model, r_key, shape, learning_rate ) -> train_state.TrainState:
    print(shape)
    init_variables = model.init(r_key, jnp.ones(shape))  # Initialize the Model
    optimizer = optax.adam(learning_rate) # Create the optimizer
    # Create a State
    return train_state.TrainState.create(
        apply_fn = model.apply,
        tx=optimizer,
        params=init_variables['params']
    )

learning_rate = 1e-4
batch_size_no = 64

model = MLPModel() # Instantiate the Model

key, rng = jax.random.split(jax.random.PRNGKey(0))
x = jnp.empty((batch_size_no, 28, 28, 1)) # Dummy Input
_, image_height, image_width, channels = x.shape
##########################################<< CREATING MODEL STATE>>#########################################

state = init_train_state( model, rng, (image_height * image_width, channels), learning_rate )
state
print("state initiated ✅")

##########################################<< MODEL LOSS METRICS FOR GRADIENT COMPUTATION >>#########################################
def image_difference_loss(logits, labels):
    loss = .5 * jnp.mean((logits - labels) ** 2)
    return loss
def compute_metrics(*, logits, labels):
  loss = image_difference_loss(logits, labels)
  loss = lax.pmean(loss, axis_name="batch");print("ok4")
  metrics = {
      'loss': loss,
      'logits': logits, #PREDICTED IMAGE
      'labels': labels  #ACTUAL IMAGE
  }
  return metrics

##########################################<< MODEL TRAINING STEP FOR UPDATING WEIGHTS FROM CALCULATED GRAGIENTS FROM CHANGING MODEL STATE >>#########################################
import jax

def train_step(state: train_state.TrainState, batch: jnp.asarray, rng):
    print(batch)
    image, label = batch
    print(image,"<<<image")
    print(label,"<<<label")    
    def loss_fn(params):
        logits = state.apply_fn({'params': params}, image);print("done1",logits.shape)
        loss =  image_difference_loss(logits, label);print("done2",loss.shape)
        return loss, logits

    print("ok1really")
    gradient_fn = jax.value_and_grad(loss_fn, has_aux=True);print("ok1")
    (_, logits), grads = gradient_fn(state.params);print("ok2")
    #train_loss, gradients_each = jax.value_and_grad(loss_fn)(state.params);print("ok3")
    grads = lax.pmean(grads,"batch");print("ok4")
    # grads = jnp.mean(grads);print("ok4")
    state = state.apply_gradients(grads=grads);print("ok5")
    # train_loss = jnp.mean(train_loss);print("ok6")
    logs = compute_metrics(logits=logits, labels=label);print("ok7")
    return state, logs

parallel_train_step = jax.pmap(train_step, "batch")
# parallel_train_step = jax.pmap(train_step, axis_name="batch", in_axes = (0, 0, 0))

import jax
@jax.jit
def eval_step(state, batch):
    image, label = batch
    logits = state.apply_fn({'params': state.params}, image)
    return compute_metrics(logits=logits, labels=label)

##########################################<< DATASET GENERATING FUNCTION HELPER IN GRB TO GRAY CONVERSION >>#########################################
newsize = (150, 150) # /.... 233 * 454
from PIL import Image
import jax.numpy as jnp

def imageGRAY(argv):
    im = Image.open(argv).convert('L')
    tvt, tvu = jnp.asarray(im.resize(newsize)),jnp.asarray(im.resize(newsize)).reshape(-1,1)
    return tvt, tvu
def imageRGB(argv):
    im = Image.open(argv)
    tvt, tvu = jnp.asarray(im.resize(newsize)),jnp.asarray(im.resize(newsize)).reshape(-1,3)
    return tvt, tvu


##########################################<< ANALYSING DATASETS FOR BATCH DISTRIBUTION >>#########################################
# testing batching of the dataset frfom the total dataset, by assuming batch size as 8 then running 50 epochs over the batch , then moving processing onto next batch>>>
batch_size = jax.device_count()    #///// batchsize depending on number of devices available for processing
import os
image_dir = r'/content/flowers/all/'
#############################################################################
prefix = "sur_refl_"
bandend = ["c",".jpg", "b02", "b03", "b04", "b05", "b06", "b07", "day_of_year", "qc_500m", "raz", "state_500m", "szen", "vzen"]
DayOY = "_doy\[0-9]+_aid0001"
fileExt = r'.jpg'
expression_b2 = bandend[1]
total_images =  [f for f in os.listdir(image_dir) if f.__contains__(expression_b2)]
total_images.sort()
total_images_path = [os.path.join(image_dir, i) for i in total_images if i != 'outputs']
no_of_batches = int(len(total_images_path)/batch_size)



##########################################<< RUNNING TRAINNING >>#########################################
#@title # **👠HIGH HEELS RUN >>>>>>>>>>>** { vertical-output: true }
newsize = (260, 260) # /.... 233 * 454

import jax
from jax import random
def batchedimages(image_locations):
  ddyss = jnp.asarray((imageRGB(total_images_path[image_locations[0]])[1],
                      imageRGB(total_images_path[image_locations[1]])[1],
                      imageRGB(total_images_path[image_locations[2]])[1],
                      imageRGB(total_images_path[image_locations[3]])[1],
                      imageRGB(total_images_path[image_locations[4]])[1],
                      imageRGB(total_images_path[image_locations[5]])[1],
                      imageRGB(total_images_path[image_locations[6]])[1],
                      imageRGB(total_images_path[image_locations[7]])[1]))
  ddxss = jnp.asarray((imageGRAY(total_images_path[image_locations[0]])[1],
                      imageGRAY(total_images_path[image_locations[1]])[1],
                      imageGRAY(total_images_path[image_locations[2]])[1],
                      imageGRAY(total_images_path[image_locations[3]])[1],
                      imageGRAY(total_images_path[image_locations[4]])[1],
                      imageGRAY(total_images_path[image_locations[5]])[1],
                      imageGRAY(total_images_path[image_locations[6]])[1],
                      imageGRAY(total_images_path[image_locations[7]])[1]))
  #print(ddyss.shape,"<<<< ddyss.shape???",ddxss.shape,"<<<< ddxss.shape???") #to check shape 
  batch_ccc = ddyss, ddxss
  return batch_ccc
  
def data_stream():
  key = random.PRNGKey(0)
  perm = random.permutation(key, len(total_images_path))
  for i in range(no_of_batches):
    batch_idx = perm[i * batch_size : (i + 1) * batch_size]; #print(batch_idx)
    yield batchedimages(batch_idx)

batches = data_stream()  ### this stream will utilize the array of paths of images to a folder, then "generate" batches into the variable

next(batches)[0].shape ### this command starts initial 8 image  stream, if callled inside a iteration loop then it will get next images for calculations>>>
vv, shapea, channels = next(batches)[0].shape # seitting values >>> [0] 8 784 3  [1] 8 784 1 ; RGB & GRAYSCALE versions 8 images each converted to 1-D array
print(vv, shapea, channels)
######################<<< summary writer for tensor board
# from torch.utils.tensorboard import SummaryWriter
# logdir = "runs"
# writer = SummaryWriter(logdir)
######################
######################
rng = jax.random.PRNGKey(0)
# dropout_rngs = jax.random.split(rng, jax.local_device_count())
######################
#################################<<< checking if checkpoint already available
import os # importing os module
import re # to find file using regular expression
checkpoint_available = 0
pattern = re.compile("checkpoint_\d+")   # to search for "checkpoint_*munerical value*" numerical value of any length is denoted by regular expression "\d+"
dir = "/content/ckpts/"
isFile = os.path.isdir(dir)
if isFile:
  for filepath in os.listdir(dir):
      if pattern.match(filepath):
          checkpoint_available = 1
#################################
##########################################<<< loading checkpoint by checking the Flag available
from flax.training import checkpoints
if checkpoint_available:
  CKPT_DIR = 'ckpts'
  restored_state = checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=state)
  #state = flax.jax_utils.replicate(restored_state)
  print("true <<< File loaded for and replicated to all devices")
##########################################
######################<<<< initiating train state
count = 0
if count == 0 :
  state = init_train_state( model, rng, (shapea, channels), learning_rate ) 
  count = 1
state = flax.jax_utils.replicate(state)  # FLAX will replicate the state to every device so that updating can be made easy
dropout_rngs = jax.random.split(rng, jax.local_device_count())
######################
from flax.training import checkpoints
import time
total_epochs = 50
for epochs in range(total_epochs):   # EPOCHS for training & updating the initiated state, metrics may show the loss in each epochs or iteration
  start_time = time.time()
  batches = data_stream()  ### this stream will utilize the array of paths of images to a folder, then "generate" batches into the variable
  if checkpoint_available:
    CKPT_DIR = 'ckpts'
    restored_state = checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=state)
    state = restored_state
    checkpoint_available = 0 # << Flag updated >>> to stop loading the same checkpoint in the next iteration then remove the checkpoint directory
    !rm -r /content/ckpts
  for bbb in range(no_of_batches-5):
    print(bbb,"of total number of batches",no_of_batches)
    state, metrics = parallel_train_step(state, next(batches), dropout_rngs)
    print("<<✅✅✅epoc : ",epochs," complete✅✅✅>>\n",metrics['loss'][0]) #printing loss 1 out of 8 processed in 8 devices
    
    #############################################<<< output visualization 👍🏻
    from google.colab.patches import cv2_imshow
    import numpy as np        # source : https://www.folkstalk.com/tech/how-to-convert-numpy-array-to-cv2-image-with-code-examples/
    
    print("logits shape ⚡⚡⚡⚡", metrics['logits'][0].shape)
    L1 = metrics['logits'][0]
    # predicted_image = np.array(L1,  dtype=np.uint8).reshape(newsize) # This would be your image array
    # cv2img = cv2_imshow(predicted_image) # This work the same as passing an image
    for i in range(0,metrics['labels'].shape[0]):
      print(i)
      predicted_image = np.array(metrics['logits'][i],  dtype=np.uint8).reshape(newsize) # This would be your image array
      cv2_imshow(predicted_image)

      print("labels shape ⚡⚡⚡⚡", metrics['labels'][0].shape)
      L2 = metrics['labels'][0]
      actual_image = np.array(metrics['labels'][i],  dtype=np.uint8).reshape(newsize) # This would be your image array newsize = image_width , image_height
      cv2img = cv2_imshow(actual_image) # This work the same as passing an image
    #############################################<<< output visualization 👍🏻
    

    

    writer.add_scalar('Loss', int(metrics['loss'][0]), epochs)
  epoch_time = time.time() - start_time
  print(f"Epoch {epochs} in {epoch_time:0.2f} sec")
  ##################################################<<< model saving mechanism for flax model state as checkpoints for each epochs,"checkpoint" is a terminology that means all the model weights and biases during the calculation till the completion of 1 epoch were being updated, then this final set of weights and biases including their placement inside the model will be saves as a (schema+weight values) saved as checkpoint in the mentioned <<CKPT_DIR = 'ckpts'>> mentioned folder.  
  CKPT_DIR = 'ckpts'
  checkpoints.save_checkpoint(ckpt_dir=CKPT_DIR, target=state, step= epochs)     # naming of the checkpoint is "checkpoint_*"  where "*" => value of the steps variable, i.e. 'epochs'
  restored_state = checkpoints.restore_checkpoint(ckpt_dir=CKPT_DIR, target=state) # using to get the checkpoint loaded , it can be latest one , or if already available as checkpoint in the "CKPT_DIR" directory then take the file from directory then save in >> restored_checkpoints
  ##################################################
  # images = total_images_path[batch_idx]
writer.flush()

 